In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

from mle_datathon.utils import get_abs_path, load_config, set_log

from mle_datathon.data_processing.feature_engineering import transform_new_data
import requests
import json

import pandas as pd

logger = set_log(level=50, logfile="mle_datathon")
url = "http://localhost:8000/api/v1/model/predict"

local_path = os.getcwd()
config = load_config(local_path)

paths = config["paths"]

# Todos os dados que entrarem na aplicação vão passar passar pelo pipeline de limpeza e construção das camadas do Data Lake.
# Então nós já vamos pegar da camada silves pra otimizar
df_prospects = pd.read_parquet(get_abs_path(local_path, paths["prospects_silver"]))
df_vagas = pd.read_parquet(get_abs_path(local_path, paths["vagas_silver"]))

[nltk_data] Downloading package stopwords to /home/felip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean_features_data(df):
    colunas_remover = [
        "analista_responsavel",
        "cidade",
        "cliente",
        "cod_vaga",
        "codigo",
        "data_candidatura",
        "data_final",
        "data_inicial",
        "data_requicisao",
        "empresa_divisao",
        "estado",
        "limite_esperado_para_contratacao",
        "local_trabalho",
        "nome",
        "recrutador",
        "regiao",
        "requisitante",
        "situacao_candidado",
        "solicitante_cliente",
        "ultima_atualizacao",
        "tipo_contratacao",
        "nivel_academico",
    ]
    colunas_remover_2 = [
        "titulo",
        "comentario",
        "titulo_vaga",
        "prazo_contratacao",
        "prioridade_vaga",
        "nivel profissional",
        "nivel_ingles",
        "nivel_espanhol",
        "areas_atuacao",
        "principais_atividades",
        "competencia_tecnicas_e_comportamentais",
        "demais_observacoes",
        "equipamentos_necessarios",
        "habilidades_comportamentais_necessarias",
        "valor_venda",
        "valor_compra_1",
    ]

    colunas_remover.extend(colunas_remover_2)

    df = df.drop(columns=colunas_remover)

    features = [
        col
        for col in df.columns
        if col != "target" and pd.api.types.is_numeric_dtype(df[col])
    ]
    X = df[features]

    return X

In [3]:
um_prospect_aleatorio = df_prospects.sample(1)

um_caso_aleatorio = um_prospect_aleatorio.merge(
    df_vagas, on="cod_vaga", how="left", suffixes=("", "_vaga")
)

encoders_path = "/home/felip/projetos/datathon-mle/Datathon Decision/4_gold/encoders"

um_caso_aleatorio = transform_new_data(um_caso_aleatorio, encoders_path=encoders_path)

um_caso_aleatorio = clean_features_data(um_caso_aleatorio)

payload = json.dumps(
    {
        "model_name": "RandomForest",
        "model_version": 1,
        "data": um_caso_aleatorio.values.tolist(),
    }
)
headers = {"Content-Type": "application/json", "Content-Type": "application/json"}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

2025-05-20 21:25:08,410 - feature_engineering - INFO - [Inferência] Carregando encoders salvos...
2025-05-20 21:25:08,422 - feature_engineering - INFO - [Inferência] Aplicando transformação de nivel_academico...
2025-05-20 21:25:08,435 - feature_engineering - INFO - [Inferência] Aplicando transformação de tipo_contratacao...


2025-05-20 21:25:08,440 - feature_engineering - INFO - [Init] Carregando modelo de embeddings...
2025-05-20 21:25:08,592 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-05-20 21:25:08,595 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
2025-05-20 21:25:12,960 - feature_engineering - INFO - [principais_atividades] Criando features de tamanho e palavras...
2025-05-20 21:25:12,968 - feature_engineering - INFO - [principais_atividades] Criando embeddings agregados...
2025-05-20 21:25:12,976 - feature_engineering - INFO - [Embeddings] Iniciando geração para 1 textos...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-20 21:25:13,577 - feature_engineering - INFO - [Embeddings] Geração finalizada. Criando features agregadas...
2025-05-20 21:25:13,581 - feature_engineering - INFO - [principais_atividades] Features de embeddings agregados adicionadas.
2025-05-20 21:25:13,585 - feature_engineering - INFO - [competencia_tecnicas_e_comportamentais] Criando features de tamanho e palavras...
2025-05-20 21:25:13,594 - feature_engineering - INFO - [competencia_tecnicas_e_comportamentais] Criando embeddings agregados...
2025-05-20 21:25:13,601 - feature_engineering - INFO - [Embeddings] Iniciando geração para 1 textos...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 1/1 [00:00<00:00,  8.53it/s]


[[0.6925,0.3075]]


[[0.5934848484848485,0.4065151515151515]]
